## 1nn versus RandomForest

В этом задании будет использоваться датасет digits из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.

In [56]:
import pandas as pd

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KDTree
from sklearn.ensemble import RandomForestClassifier

In [3]:
digits = load_digits()
features = digits.data
target = digits.target

In [5]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, shuffle=False)


#### Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeighborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.

In [10]:
features_test[0]

array([ 0.,  0.,  7., 16., 16., 14.,  0.,  0.,  0.,  0., 16., 12., 10.,
       15.,  1.,  0.,  0.,  0., 10.,  4., 16., 10.,  0.,  0.,  0.,  0.,
        0.,  9., 16., 11.,  1.,  0.,  0.,  0.,  0.,  0.,  7., 16.,  8.,
        0.,  0.,  0.,  0.,  0.,  0., 16.,  7.,  0.,  0.,  0.,  8.,  4.,
       10., 15.,  2.,  0.,  0.,  0., 12., 16., 16.,  6.,  0.,  0.])

In [9]:
features

array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., 10.,  0.,  0.],
       [ 0.,  0.,  0., ..., 16.,  9.,  0.],
       ...,
       [ 0.,  0.,  1., ...,  6.,  0.,  0.],
       [ 0.,  0.,  2., ..., 12.,  0.,  0.],
       [ 0.,  0., 10., ..., 12.,  1.,  0.]])

In [63]:
def classifier_object(obj):
    distance = []
    
    for obj_train, target in zip(features_train, target_train):
        local_distance = 0
        
        for value_train, value_test in zip(obj_train, obj):
            local_distance += ((value_train - value_test) ** 2) ** (1/2)
            
        distance.append([local_distance, target])
    
    distance = sorted(distance)
    
    return distance[0][1]

In [64]:
result_prediction = []

for obj_test in features_test:
    result_prediction.append(classifier_object(obj_test))

In [65]:
result_prediction

[3,
 7,
 3,
 3,
 4,
 6,
 6,
 6,
 4,
 9,
 1,
 5,
 0,
 9,
 6,
 2,
 8,
 2,
 0,
 0,
 1,
 7,
 6,
 3,
 2,
 1,
 7,
 4,
 6,
 3,
 1,
 3,
 9,
 1,
 7,
 6,
 8,
 4,
 3,
 1,
 4,
 0,
 5,
 3,
 6,
 9,
 6,
 1,
 7,
 5,
 4,
 4,
 7,
 2,
 8,
 2,
 2,
 5,
 7,
 9,
 5,
 4,
 8,
 8,
 4,
 9,
 0,
 8,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 0,
 9,
 5,
 5,
 6,
 5,
 0,
 9,
 8,
 9,
 8,
 4,
 1,
 7,
 7,
 3,
 5,
 1,
 0,
 0,
 2,
 2,
 7,
 8,
 2,
 0,
 1,
 2,
 6,
 3,
 3,
 7,
 3,
 3,
 4,
 6,
 6,
 6,
 4,
 9,
 1,
 5,
 0,
 9,
 5,
 2,
 8,
 2,
 0,
 0,
 1,
 7,
 6,
 3,
 2,
 1,
 7,
 4,
 6,
 3,
 1,
 3,
 9,
 1,
 7,
 6,
 8,
 4,
 3,
 1,
 4,
 0,
 5,
 3,
 6,
 9,
 6,
 1,
 7,
 5,
 4,
 4,
 7,
 2,
 8,
 2,
 2,
 5,
 7,
 9,
 5,
 4,
 8,
 8,
 4,
 9,
 0,
 9,
 9,
 8,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 1,
 9,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 1,
 9,
 0,
 9,
 5,
 5,
 6,
 5,
 0,
 9,
 1,
 5,
 8,
 4,
 1,
 7,
 7,
 3,
 5,
 1,
 0,
 0,
 2,
 2,
 7,
 8,


In [66]:
target_test

array([3, 7, 3, 3, 4, 6, 6, 6, 4, 9, 1, 5, 0, 9, 5, 2, 8, 2, 0, 0, 1, 7,
       6, 3, 2, 1, 7, 4, 6, 3, 1, 3, 9, 1, 7, 6, 8, 4, 3, 1, 4, 0, 5, 3,
       6, 9, 6, 1, 7, 5, 4, 4, 7, 2, 8, 2, 2, 5, 7, 9, 5, 4, 8, 8, 4, 9,
       0, 8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9,
       0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 9, 5, 5, 6, 5, 0, 9, 8, 9, 8, 4,
       1, 7, 7, 3, 5, 1, 0, 0, 2, 2, 7, 8, 2, 0, 1, 2, 6, 3, 3, 7, 3, 3,
       4, 6, 6, 6, 4, 9, 1, 5, 0, 9, 5, 2, 8, 2, 0, 0, 1, 7, 6, 3, 2, 1,
       7, 4, 6, 3, 1, 3, 9, 1, 7, 6, 8, 4, 3, 1, 4, 0, 5, 3, 6, 9, 6, 1,
       7, 5, 4, 4, 7, 2, 8, 2, 2, 5, 7, 9, 5, 4, 8, 8, 4, 9, 0, 8, 9, 8,
       0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 0, 1, 2, 3, 4, 5, 6, 9, 0, 1, 2, 3,
       4, 5, 6, 7, 8, 9, 0, 9, 5, 5, 6, 5, 0, 9, 8, 9, 8, 4, 1, 7, 7, 3,
       5, 1, 0, 0, 2, 2, 7, 8, 2, 0, 1, 2, 6, 3, 3, 7, 3, 3, 4, 6, 6, 6,
       4, 9, 1, 5, 0, 9, 5, 2, 8, 0, 1, 7, 6, 3, 2, 1, 7, 4, 6, 3, 1, 3,
       9, 1, 7, 6, 8, 4, 3, 1, 4, 0, 5, 3, 6, 9, 6,

In [67]:
1 - accuracy_score(result_prediction, target_test)

0.0444444444444444

In [70]:
with open('1_1nn_vs_randomforest.txt', 'w') as f:
    f.write('0.0444444444444444')

#### Задание 2

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [68]:
model = RandomForestClassifier(n_estimators=1000)
model.fit(features_train, target_train)
predicted_values = model.predict(features_test)

In [69]:
1 - accuracy_score(predicted_values, target_test)

0.06888888888888889

In [71]:
with open('2_1nn_vs_randomforest.txt', 'w') as f:
    f.write('0.06888888888888889')